<a href="https://colab.research.google.com/github/mike-papa/big_data/blob/main/Big_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark==1.3.0
!pip install -q pyspark==3.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


Zadanie 4 <br>
Załaduj plik pracownicy.csv z poprawnie zdefiniowaną strukturą i typami danych (schema)
Dla każdego roku urodzenia występującego w danych wylicz średnią stawkę godzinową
posortuj od najmłodszych lat do najstarszych
Wynik zapisz do excela

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql.functions import col
from pyspark.sql.functions import avg


# Inicjalizacja Spark Session
spark = SparkSession.builder.appName("Przetwarzanie danych pracowników").getOrCreate()

# Ścieżka do pliku CSV
file_path = 'pracownicy.csv'

# Definicja schematu
schema = StructType([
    StructField("imie", StringType(), True),
    StructField("nazwisko", StringType(), True),
    StructField("stawka_godzinowa_w_dolarach", FloatType(), True),
    StructField("rok_urodzenia", IntegerType(), True)
])

# Załadowanie danych
df = spark.read.csv(file_path, schema=schema, header=True, sep=';')

df.show(20)

# Obliczanie średniej stawki godzinowej dla każdego roku urodzenia
avg_stawka_df = df.groupBy("rok_urodzenia").agg(avg("stawka_godzinowa_w_dolarach").alias("średnia_stawka_godzinowa")).orderBy("rok_urodzenia")
avg_stawka_df.show()
# Konwersja do Pandas DataFrame
result_df = avg_stawka_df.toPandas()

# Zapis do pliku Excel
result_df.to_excel("srednie_stawki_godzinowe.xlsx", index=False)

# Zamknięcie Spark Session
spark.stop()

+---------+---------+---------------------------+-------------+
|     imie| nazwisko|stawka_godzinowa_w_dolarach|rok_urodzenia|
+---------+---------+---------------------------+-------------+
|   Maciej| Kowalski|                       78.0|         1944|
|  Michael| Kowalski|                       57.0|         1965|
|      Ewa| Kowalska|                       23.0|         1999|
|     John|    Nowak|                       12.0|         2020|
|    Steve| Stevniak|                       67.0|         1955|
|     Bill|    Gates|                       67.0|         1955|
|     Jan | Kowalski|                       23.0|         1999|
|     John|    Nowak|                       12.0|         2020|
|    Steve| Stevniak|                       67.0|         1955|
|     Bill|    Gates|                       67.0|         1955|
|      Jan| Kowalski|                       23.0|         1999|
|     John|    Nowak|                       12.0|         2020|
|    Steve|Stevniak |                   

Zadanie 5 <br>
Jakiego dnia ile było wejść a angielską wikipedię? Rozwiąż na dwa sposoby (wykorzystując czyste RDD oraz Dataframes). 4 kolumna w danych oznacza ile bylo wejsc. <br>
Plik : pagecounts

In [ ]:
from pyspark import SparkContext

sc = SparkContext.getOrCreate()
files = "pagecounts/part-*"  # Ścieżka do plików danych
rdd = sc.textFile(files)

def process_line(line):
    parts = line.split()
    date = parts[0][:8] # wyciąganie daty z timestamp
    lang = parts[1] # język
    page_views = int(parts[3]) # liczba wyświetleń strony
    return (date, lang, page_views)

# Przetwarzanie linii
rdd_processed = rdd.map(process_line)

for record in rdd_processed.take(5):
    print(record)

# Filtrowanie tylko wpisów dla 'en'
rdd_en = rdd_processed.filter(lambda x: x[1] == 'en')

for record in rdd_en.take(5):
    print(record)

# Agregacja liczby wejść dla każdego dnia
rdd_aggregated = rdd_en.map(lambda x: (x[0], x[2])).reduceByKey(lambda x, y: x + y)

# Wyświetlenie wyników
print(rdd_aggregated.collect())

('20090505', 'aa', 2)
('20090505', 'ab', 1)
('20090505', 'ab', 1)
('20090505', 'af.b', 1)
('20090505', 'af.d', 4)
('20090505', 'en', 4)
('20090505', 'en', 21)
('20090505', 'en', 9)
('20090505', 'en', 2)
('20090505', 'en', 4)
[('20090505', 7076855), ('20090507', 6175726)]


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum

# Inicjalizacja SparkSession
spark = SparkSession.builder.appName("Wikipedia Page Views").getOrCreate()

# Wczytanie danych do DataFrame
df = spark.read.csv("pagecounts/part-*", sep=" ").toDF("timestamp", "lang", "title", "page_views", "size")

df.show(5)

# Filtrowanie danych dla języka angielskiego
df_en = df.filter(col("lang") == "en")

df_en.show(5)

# Grupowanie danych po dacie i sumowanie liczby wejść
df_aggregated = df_en.groupBy(df_en.timestamp.substr(1, 8).alias("date")).agg(sum("page_views").alias("total_views"))

# Wyświetlenie wyników
df_aggregated.show()

# Zamykanie Spark Session
spark.stop()


+---------------+----+--------------------+----------+------+
|      timestamp|lang|               title|page_views|  size|
+---------------+----+--------------------+----------+------+
|20090505-000000|  aa|           Main_Page|         2|  9980|
|20090505-000000|  ab|%D0%90%D0%B8%D0%B...|         1|   465|
|20090505-000000|  ab|%D0%98%D1%85%D0%B...|         1| 16086|
|20090505-000000|af.b|            Tuisblad|         1| 36236|
|20090505-000000|af.d|            Tuisblad|         4|189738|
+---------------+----+--------------------+----------+------+
only showing top 5 rows

+---------------+----+--------------------+----------+------+
|      timestamp|lang|               title|page_views|  size|
+---------------+----+--------------------+----------+------+
|20090505-000000|  en|                   !|         4|170494|
|20090505-000000|  en|                 !!!|        21|306957|
|20090505-000000|  en|      !!!Fuck_You!!!|         9| 87025|
|20090505-000000|  en|!!!Fuck_You!!!_an...|  

Zadanie 6 <br>
Wczytaj wiki_sample.parquet (moodle)
Odpowiedz na pytanie którzy użytkownicy edytowali najwięcej stron na wikipedii (top 5)?
Wykonaj zadanie na dwa sposoby (Dataframe SQL oraz Dataframe programmatic API)

In [ ]:
#brak pliku wiki_sample.parquet na moodle